# Link del video yt
https://www.youtube.com/watch?v=kCc8FmEb1nY

# Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
!pip install datasets
!pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Model

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchtext.data.utils import get_tokenizer

from datasets import load_dataset
from transformers import DistilBertTokenizer

from tqdm import tqdm

from random import randrange
import pandas as pd
import csv



batch_size = 16       # How many independent sequences will we process in parallel?
block_size = 32       # What is the maximum context length for predictions? Increase for had bigger phrases ! 
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 50      # 200
n_embd = 64
n_head = 4
# TODO: increase number of blocks !  
n_layer = 50           # Number of layers
dropout = 0.0

max_iters = 100     # 10000
model_name = 'model_nn_' + str(n_layer) + '_gen_ ' + str(max_iters) +'.pt'
csv_file_name = 'GFG' + str(max_iters) + '.csv'
# ------------
default_str = " "
df_loss_train = []
df_loss_test = []
df_step = []
fields = ['train_loss', 'loss_test', 'step'] 


# Load datasets
dataset = load_dataset("vicgalle/alpaca-gpt4", split="train")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


torch.manual_seed(1337)
vocab_size = tokenizer.vocab_size
print(f"device: {device}")

device: cpu


In [4]:
# encode = lambda s: [stoi[c] if c in stoi else stoi[default_str] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
# Carica il tokenizer preaddestrato di BERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
encode = lambda s: tokenizer.encode_plus(s, padding=True, return_tensors='pt', add_special_tokens=True, max_length=block_size + 1, truncation=True)['input_ids'].flatten().tolist()
decode = lambda s: tokenizer.decode(s)




def get_batch(split):
  data = []
  while(len(data) < block_size + 1):
    item = randrange(dataset.num_rows)
    if split == 'train':
      data = encode( dataset['instruction'][item] if dataset['input'][item] == "" else dataset['instruction'][item] + " " + dataset['input'][item] ) #
    else:
      data = encode( dataset['output'][item] )

  data = torch.tensor(data, dtype=torch.long)

  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)

  return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Train

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
rows = [] 

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# for iter in range(max_iters):
for iter in tqdm (range (max_iters), desc="Loading..."):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(f"\tStep {iter}: \ttrain loss {losses['train']:.4f}, val loss {losses['val']:.4f}  ")
        df_loss_train.append( round(losses['train'].item(), 4) )
        df_loss_test.append( round(losses['val'].item(), 4) )
        df_step.append( iter )

        rows.append([round(losses['train'].item(), 4), round(losses['val'].item(), 4), iter])
    
    # sample a batch of data
    xb, yb = get_batch('train')


    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Update bar
    pass

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(f"\nText: {decode(m.generate(context, max_new_tokens=50)[0].tolist())}")


# Save datas
torch.save(m.state_dict(), model_name)
with open(csv_file_name, 'w') as f:
    write = csv.writer(f)
     
    write.writerow(fields)
    write.writerows(rows)

4.138682 M parameters


Loading...:  90%|████████████████████████████████████████████████████▎     | 90290/100000 [78:33:08<6:03:56,  2.25s/it]

## Loss

In [ ]:
import pandas as pd

df = pd.DataFrame({'train_loss': df_loss_train, 'loss_test': df_loss_test})
df.plot(df['step'], y=['train_loss','loss_test'])
# df.describe()

## Execution
This version is from running model

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'


context = torch.zeros((1, 1), dtype=torch.long, device=device)
inptVal = torch.as_tensor(encode("How are you ?"), dtype=torch.long, device=device)

"""
print(inptVal.size())
print(context.size())
print(context)
print(inptVal)
print(inptVal.size()[0])
print("\n\n\n\n")
"""

print(decode(m.generate(inptVal.view(1, inptVal.size()[0]), max_new_tokens=50)[0].tolist()))
# print(decode(m.generate(context, max_new_tokens=50)[0].tolist()))

# Execution
This version is from a saved model

In [ ]:
import torch



context = torch.zeros((1, 1), dtype=torch.long, device=device)
inptVal = torch.as_tensor(encode("How are you ?"), dtype=torch.long, device=device)


model = BigramLanguageModel()
model.load_state_dict(torch.load('./model_m_scripted_5000.pt', map_location=torch.device(device)))
m = model.to(device)


print(decode(m.generate(inptVal.view(1, inptVal.size()[0]), max_new_tokens=50)[0].tolist()))
print(decode(m.generate(context, max_new_tokens=50)[0].tolist()))

In [ ]:
import pandas as pd

df = pd.read_csv('GFG.csv')


# Display loss
df.plot(df['step'], y=['train_loss','loss_test'])
# df.describe()